http://host.robots.ox.ac.uk/pascal/VOC/voc2007/

In [1]:
import os
from datetime import datetime

import tensorflow as tf
from tensorflow import keras

from centernet import CenterNet
from data import VOCDataset
from losses import centernet_loss
from callbacks import VisCallback

# imgaug : conda install imgaug

In [2]:
# GPU list load
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
"""
data_path = "E:\github2\centernet-keras\VOCdevkit\VOC2007"
train_file = "VOCdevkit\VOC2007\ImageSets\Main\\train.txt"
val_file = "VOCdevkit\VOC2007\ImageSets\Main\\val.txt"

log_path = "./logs/test"
"""

'\ndata_path = "E:\\github2\\centernet-keras\\VOCdevkit\\VOC2007"\ntrain_file = "VOCdevkit\\VOC2007\\ImageSets\\Main\\train.txt"\nval_file = "VOCdevkit\\VOC2007\\ImageSets\\Main\\val.txt"\n\nlog_path = "./logs/test"\n'

In [4]:
print(VOCDataset)

<class 'data.VOCDataset'>


In [5]:
def main():
    # load dataset
    train_dataset_raw = VOCDataset(data_path, input_shape, train_file,
                                   batch_size, True)
    train_dataset = train_dataset_raw.load_dataset()

    val_dataset_raw = VOCDataset(data_path, input_shape, train_file,
                                 batch_size, False)
    val_dataset = val_dataset_raw.load_dataset()

    vis_dataset_raw = VOCDataset(data_path, input_shape, train_file, 1, False)
    vis_dataset = vis_dataset_raw.load_dataset().repeat()

    steps_per_epoch = len(train_dataset_raw) // batch_size
    print(steps_per_epoch)

    # callbacks
    logdir = os.path.join(log_path, datetime.now().strftime("%Y%m%d-%H%M%S"))
    tb_callback = keras.callbacks.TensorBoard(logdir, update_freq=100)
    ckpt_callback = keras.callbacks.ModelCheckpoint(
        filepath=logdir,
        save_best_only=True,
        monitor="val_loss",
        verbose=1,
    )
    vis_callback = VisCallback(vis_dataset,
                               class_names=train_dataset_raw.class_names,
                               logdir=logdir,
                               update_freq=100)

    # load model
    model = CenterNet(train_dataset_raw.class_names,
                      backbone_weights='imagenet',
                      freeze=freeze,
                      finetune=finetune)

    if ckpt_path:
        model.load_weights(ckpt_path)
        print(f"Loding pretrained weights from {ckpt_path} !")

    # model compile
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=lr,
        decay_steps=steps_per_epoch,
        decay_rate=0.94,
        staircase=True)
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    model.compile(optimizer=optimizer, loss=centernet_loss, run_eagerly=False)

    
    model.fit(x=train_dataset,
              validation_data=val_dataset,
              epochs=epochs,
              callbacks=[tb_callback, ckpt_callback, vis_callback])

In [7]:
if __name__ == "__main__":
    model_path = ''
    input_shape = (512, 512)
    backbone = 'resnet50'

    epochs = 1
    batch_size = 1
    buffer_size = batch_size * 5
    lr = 1e-2
    freeze = True
    finetune = True or freeze
    ckpt_path = ""
    # ckpt_path =  "./logs/test/20220417-140619"

    #data_path = "..\\..\\Centernet\\VOC2007"
    data_path = "..\\..\\Centernet\\VOC2007"
    #train_file = "..\\..\\Centernet\\VOC2007\\ImageSets\\Main\\train.txt"
    #val_file = "..\\..\\Centernet\\VOC2007\\ImageSets\\Main\\val.txt"
    
    train_file = "..\\..\\Centernet\\VOC2007\\ImageSets\\ceph\\train.txt"
    val_file = "..\\..\\Centernet\\VOC2007\\ImageSets\\ceph\\val.txt"

    log_path = "./logs/test"

    main()

loading the context of data_file ...: 5it [00:00, 8514.62it/s]
loading the context of data_file ...: 5it [00:00, ?it/s]
loading the context of data_file ...: 5it [00:00, 689.04it/s]


5
      1/Unknown - 0s 0s/step - loss: 57944.7305 - hm_loss: 57942.2734 - wh_loss: 0.0000e+00 - reg_loss: 0.0000e+00WARNING:tensorflow:From C:\Users\Cyan ovo\anaconda3\envs\keras\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
      5/Unknown - 3s 697ms/step - loss: 26402.4984 - hm_loss: 26400.3465 - wh_loss: 0.0000e+00 - reg_loss: 0.0000e+00
Epoch 00001: val_loss improved from inf to 2.90394, saving model to ./logs/test\20220524-215013
5/5 [==============================] - 9s 2s/step - loss: 23024.2134 - hm_loss: 23022.0210 - wh_loss: 0.0000e+00 - reg_loss: 0.0000e+00 - val_loss: 2.9039 - val_hm_loss: 0.0912 - val_wh_loss: 0.0000e+00 - val_reg_loss: 0.0000e+00
